In [1]:
from datetime import datetime

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

import patsy


from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report

from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score

from scipy.stats import ks_2samp

In [2]:
#  Abrindo o dataset SSA
df = pd.read_excel(r"C:\Users\sabri\SSATeams.xlsx")
df.head()

,replay_id,replay_title,map,date,result,team_name,opposing_team_name,score,goals,assists,saves,shots,shots_conceded,shooting_percentage,demos_inflicted,demos_taken,save_conceded
0,000bb4b3-78e2-4886-a3df-7f417e1bbe78,SSA [H] [BRO] vs [AE] G5 2022-10-14.17.37,Champions Field,2022-10-14,win,THE BROS,AUREON ESPORTS,868,1,1,3,5,5,20.000000,3,1,0.600000
1,000bb4b3-78e2-4886-a3df-7f417e1bbe78,SSA [H] [BRO] vs [AE] G5 2022-10-14.17.37,Champions Field,2022-10-14,loss,AUREON ESPORTS,THE BROS,764,0,0,4,5,5,0.000000,1,3,0.800000
2,009b96ef-ffb5-4218-a4d4-b19a39ef5362,SSA I LMT vs NB G3 2023-02-10.18.03,DFH Stadium,2023-02-10,win,LIMITLESS,NKOSI BALLERS,1248,4,4,1,15,2,26.666667,3,1,0.500000
3,009b96ef-ffb5-4218-a4d4-b19a39ef5362,SSA I LMT vs NB G3 2023-02-10.18.03,DFH Stadium,2023-02-10,loss,NKOSI BALLERS,LIMITLESS,727,0,0,6,2,15,0.000000,1,3,0.400000
4,00c462b9-897e-4cd3-bd49-8bdde2e14440,SSA I LMT vs ESPI G2 2023-01-27.17.51,Forbidden Temple,2023-01-27,loss,ESPIONAGE,LIMITLESS,690,1,0,3,2,13,50.000000,0,4,0.230769


In [3]:
#  Renomear a coluna assists para goals_assists e goals para goals_solo
df.rename(columns={'assists':'goals_assists'}, inplace = True)
df.rename(columns={'goals':'goals_solo'}, inplace = True)
df

,replay_id,replay_title,map,date,result,team_name,opposing_team_name,score,goals_solo,goals_assists,saves,shots,shots_conceded,shooting_percentage,demos_inflicted,demos_taken,save_conceded
0,000bb4b3-78e2-4886-a3df-7f417e1bbe78,SSA [H] [BRO] vs [AE] G5 2022-10-14.17.37,Champions Field,2022-10-14,win,THE BROS,AUREON ESPORTS,868,1,1,3,5,5,20.000000,3,1,0.600000
1,000bb4b3-78e2-4886-a3df-7f417e1bbe78,SSA [H] [BRO] vs [AE] G5 2022-10-14.17.37,Champions Field,2022-10-14,loss,AUREON ESPORTS,THE BROS,764,0,0,4,5,5,0.000000,1,3,0.800000
2,009b96ef-ffb5-4218-a4d4-b19a39ef5362,SSA I LMT vs NB G3 2023-02-10.18.03,DFH Stadium,2023-02-10,win,LIMITLESS,NKOSI BALLERS,1248,4,4,1,15,2,26.666667,3,1,0.500000
3,009b96ef-ffb5-4218-a4d4-b19a39ef5362,SSA I LMT vs NB G3 2023-02-10.18.03,DFH Stadium,2023-02-10,loss,NKOSI BALLERS,LIMITLESS,727,0,0,6,2,15,0.000000,1,3,0.400000
4,00c462b9-897e-4cd3-bd49-8bdde2e14440,SSA I LMT vs ESPI G2 2023-01-27.17.51,Forbidden Temple,2023-01-27,loss,ESPIONAGE,LIMITLESS,690,1,0,3,2,13,50.000000,0,4,0.230769
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2367,ff462124-8e63-4448-af23-ba7523885bc0,SSA AD FISH vs Astronic G5 2022-10-14.20.26,Champions Field,2022-10-14,loss,ASTRONIC ESPORTS,FISH,758,1,0,3,6,6,16.666667,3,4,0.500000
2368,ff652d29-bf82-4a99-9719-dc8fc9408f62,SSA Y FSN vs ASTR G5 2023-06-09.20.24,Champions Field,2023-06-09,win,TEAM FUSION,ASTRONIC,1039,2,1,5,6,8,33.333333,0,3,0.625000
2369,ff652d29-bf82-4a99-9719-dc8fc9408f62,SSA Y FSN vs ASTR G5 2023-06-09.20.24,Champions Field,2023-06-09,loss,ASTRONIC,TEAM FUSION,694,0,0,3,8,6,0.000000,3,0,0.500000
2370,ffbb1db0-dc84-4f82-ada7-d049f680701b,SSA I LMT vs AIPX G2 2023-02-26.18.08,Forbidden Temple,2023-02-26,win,LIMITLESS,AIPX GAMING,747,1,0,4,5,5,20.000000,4,2,0.800000


In [4]:
#  Consertando os valores da variável goals solo para excluir os valores de goals_assists
df['goals_solo'] = df['goals_solo'] - df['goals_assists']
df

,replay_id,replay_title,map,date,result,team_name,opposing_team_name,score,goals_solo,goals_assists,saves,shots,shots_conceded,shooting_percentage,demos_inflicted,demos_taken,save_conceded
0,000bb4b3-78e2-4886-a3df-7f417e1bbe78,SSA [H] [BRO] vs [AE] G5 2022-10-14.17.37,Champions Field,2022-10-14,win,THE BROS,AUREON ESPORTS,868,0,1,3,5,5,20.000000,3,1,0.600000
1,000bb4b3-78e2-4886-a3df-7f417e1bbe78,SSA [H] [BRO] vs [AE] G5 2022-10-14.17.37,Champions Field,2022-10-14,loss,AUREON ESPORTS,THE BROS,764,0,0,4,5,5,0.000000,1,3,0.800000
2,009b96ef-ffb5-4218-a4d4-b19a39ef5362,SSA I LMT vs NB G3 2023-02-10.18.03,DFH Stadium,2023-02-10,win,LIMITLESS,NKOSI BALLERS,1248,0,4,1,15,2,26.666667,3,1,0.500000
3,009b96ef-ffb5-4218-a4d4-b19a39ef5362,SSA I LMT vs NB G3 2023-02-10.18.03,DFH Stadium,2023-02-10,loss,NKOSI BALLERS,LIMITLESS,727,0,0,6,2,15,0.000000,1,3,0.400000
4,00c462b9-897e-4cd3-bd49-8bdde2e14440,SSA I LMT vs ESPI G2 2023-01-27.17.51,Forbidden Temple,2023-01-27,loss,ESPIONAGE,LIMITLESS,690,1,0,3,2,13,50.000000,0,4,0.230769
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2367,ff462124-8e63-4448-af23-ba7523885bc0,SSA AD FISH vs Astronic G5 2022-10-14.20.26,Champions Field,2022-10-14,loss,ASTRONIC ESPORTS,FISH,758,1,0,3,6,6,16.666667,3,4,0.500000
2368,ff652d29-bf82-4a99-9719-dc8fc9408f62,SSA Y FSN vs ASTR G5 2023-06-09.20.24,Champions Field,2023-06-09,win,TEAM FUSION,ASTRONIC,1039,1,1,5,6,8,33.333333,0,3,0.625000
2369,ff652d29-bf82-4a99-9719-dc8fc9408f62,SSA Y FSN vs ASTR G5 2023-06-09.20.24,Champions Field,2023-06-09,loss,ASTRONIC,TEAM FUSION,694,0,0,3,8,6,0.000000,3,0,0.500000
2370,ffbb1db0-dc84-4f82-ada7-d049f680701b,SSA I LMT vs AIPX G2 2023-02-26.18.08,Forbidden Temple,2023-02-26,win,LIMITLESS,AIPX GAMING,747,1,0,4,5,5,20.000000,4,2,0.800000


In [5]:
#  Checando se há empates nos resultados e removendo-os
df_draw = df[df['result']=='draw']
df = df.drop(df_draw.index)

In [6]:
#  Transformando result em variável binária. Loss = 1 e win = 0
df1 = pd.get_dummies(df['result'])
#df1.head()
df1 = df1.drop(['win'], axis = 1)
#df1.head()
df = pd.concat((df, df1), axis = 1)
#df.head()
df = df.drop(['result'], axis = 1)
df = df.rename(columns = {'loss': 'result'})
df.head()

,replay_id,replay_title,map,date,team_name,opposing_team_name,score,goals_solo,goals_assists,saves,shots,shots_conceded,shooting_percentage,demos_inflicted,demos_taken,save_conceded,result
0,000bb4b3-78e2-4886-a3df-7f417e1bbe78,SSA [H] [BRO] vs [AE] G5 2022-10-14.17.37,Champions Field,2022-10-14,THE BROS,AUREON ESPORTS,868,0,1,3,5,5,20.000000,3,1,0.600000,0
1,000bb4b3-78e2-4886-a3df-7f417e1bbe78,SSA [H] [BRO] vs [AE] G5 2022-10-14.17.37,Champions Field,2022-10-14,AUREON ESPORTS,THE BROS,764,0,0,4,5,5,0.000000,1,3,0.800000,1
2,009b96ef-ffb5-4218-a4d4-b19a39ef5362,SSA I LMT vs NB G3 2023-02-10.18.03,DFH Stadium,2023-02-10,LIMITLESS,NKOSI BALLERS,1248,0,4,1,15,2,26.666667,3,1,0.500000,0
3,009b96ef-ffb5-4218-a4d4-b19a39ef5362,SSA I LMT vs NB G3 2023-02-10.18.03,DFH Stadium,2023-02-10,NKOSI BALLERS,LIMITLESS,727,0,0,6,2,15,0.000000,1,3,0.400000,1
4,00c462b9-897e-4cd3-bd49-8bdde2e14440,SSA I LMT vs ESPI G2 2023-01-27.17.51,Forbidden Temple,2023-01-27,ESPIONAGE,LIMITLESS,690,1,0,3,2,13,50.000000,0,4,0.230769,1


In [7]:
#  Excluindo várias colunas de uma vez
df.drop(['replay_id','replay_title','map','team_name','opposing_team_name', 'shots_conceded'], axis = 1, inplace = True)

In [8]:
#  Apagando as observações vazias
df.dropna(axis = 0, inplace = True)

In [9]:
#  Dividindo a base de validação
periodo_des = df[df['date'] < datetime(2023,6,1)].copy()
oot = df[df['date'] > datetime(2023,6,1)].copy()
periodo_des.reset_index(inplace = True)
oot.reset_index(inplace=True)

print(periodo_des.shape, oot.shape)

(2136, 12) (236, 12)


In [10]:
#  Definindo x e y
x_oot = oot.drop(['result', 'date', 'index'], axis = 1)
y_oot = oot['result']
x = periodo_des.drop(['result', 'date', 'index'], axis = 1)
y = periodo_des['result']
x_oot.shape

(236, 9)

In [11]:
#  Separando a base de desenvolvimento em treino e teste
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, random_state = 42)
x_train.shape, x_test.shape

((1602, 9), (534, 9))

In [12]:
#  Definindo o modelo de trabalho - Gradient Boosting

model = GradientBoostingClassifier()
model.fit(x_train, y_train)

#  Predict
# predict the mode
y_pred = model.predict(x_test)
classification_report(y_test, y_pred)
# performance evaluation metrics
print(classification_report(y_pred, y_test))

              precision    recall  f1-score   support

           0       0.90      0.86      0.88       270
           1       0.86      0.90      0.88       264

    accuracy                           0.88       534
   macro avg       0.88      0.88      0.88       534
weighted avg       0.88      0.88      0.88       534



In [13]:
#  Definindo parâmetros
param_grid = {'n_estimators': [10, 100, 300, 600] 
              , 'min_samples_leaf': [1, 10, 20, 50]
              , 'learning_rate': [.05, .1, .25, .5, 1]
             }

In [14]:
#  Separando os valores de cv
#  **parâmetro CV:** Significa *cross validation* (ou validação cruzada). 
#  Nele indicamos os índices das observações que serão utilizadas para desenvolver o modelo e para validar respectivamente. 
#  Através deste parâmetro, a função ainda consegue fazer outras modalidades de validação cruzada.

cv = KFold(n_splits=10, shuffle=True, random_state=10)

In [15]:
#  Fazendo a procura dos melhores parâmetros no modelo
grid_search = GridSearchCV(model,
                           param_grid=param_grid,
                          scoring = 'roc_auc',
                          cv = cv)

grid_search.fit(x, y)
print(grid_search.best_estimator_)

GradientBoostingClassifier(min_samples_leaf=50)


In [16]:
#  Aplicar melhores resultados aqui
model = GradientBoostingClassifier(min_samples_leaf = 50)
model.fit(x_train, y_train)


# predict the mode
y_pred = model.predict(x_test)
av1 = classification_report(y_test, y_pred)
y_pred1 = model.predict(x_oot)
av2 = classification_report(y_oot, y_pred1)
# performance evaluatio metrics
print(av1, av2)

              precision    recall  f1-score   support

           0       0.85      0.91      0.88       258
           1       0.91      0.86      0.88       276

    accuracy                           0.88       534
   macro avg       0.88      0.88      0.88       534
weighted avg       0.88      0.88      0.88       534
               precision    recall  f1-score   support

           0       0.85      0.87      0.86       118
           1       0.87      0.85      0.86       118

    accuracy                           0.86       236
   macro avg       0.86      0.86      0.86       236
weighted avg       0.86      0.86      0.86       236



In [17]:
#  Avaliando modelo na base de treino
y_pred0 = model.predict(x_train)
av0 = classification_report(y_train, y_pred0)
print(av0)

              precision    recall  f1-score   support

           0       0.95      0.94      0.94       810
           1       0.94      0.94      0.94       792

    accuracy                           0.94      1602
   macro avg       0.94      0.94      0.94      1602
weighted avg       0.94      0.94      0.94      1602



In [18]:
#  Separando as features para identificação da importância das variáveis
feature_list = list(x.columns)
#  Importância das variáveis
importances = list(model.feature_importances_)
# Print out the feature and importances 
print(feature_list, importances)

['score', 'goals_solo', 'goals_assists', 'saves', 'shots', 'shooting_percentage', 'demos_inflicted', 'demos_taken', 'save_conceded'] [0.03995655717592991, 0.12496810435883916, 0.4536635239217921, 0.0791233368938305, 0.040021616423029385, 0.04668068308678921, 0.0017057011029662746, 0.005143453077588514, 0.2087370239592349]
